# Análise de Dados 🤖
Esse notebook apresenta nossas tentivas e observações iniciais sobre o modelo de dados. 

--- 

**Conteúdo deste notebook:**
1. Imports
1. Conectando com a base
1. Separação entre resposta e atributos
1. Separação entre treino e teste
1. Aplicando nos modelos


## Imports

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier

## Conectando com a base

In [ ]:
df = pd.read_excel('../base/incluses_tratado.xlsx')

display(df.head(5))

## Separação de resposta e atributos

In [ ]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

## Pré-processamento

### Transformando colunas quantitativas em qualitativas

In [ ]:
#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

# Aplicando o preprocessador nos atributos
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())

## Definindo Pipeline incluindo SMOTE

In [ ]:
# Definir o modelo base
model = RandomForestClassifier(random_state=42)

# Definir a pipeline incluindo o SMOTE e o modelo
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42, k_neighbors=1)),
    ('classifier', model)
])

## GridSearchV

### Configurando Cross-validation 

In [ ]:
# Validação cruzada estratificada
cv = KFold(n_splits=8)

### Configurando parâmetros para o GridSearchCV

In [ ]:
# Definindo os classificadores
classifiers = {
    'knn': KNeighborsClassifier(),
    'naive_bayes': GaussianNB(),
    'decision_tree': DecisionTreeClassifier()
}

# Definindo o grid de hiperparâmetros para problemas multiclasse
param_grid = [
    {  # Parâmetros para KNeighborsClassifier
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': np.arange(1, 7, 1),
    },
    {  # Parâmetros para GaussianNB
        'classifier': [GaussianNB()],
        'classifier__var_smoothing': np.logspace(0, 100, num=50),  # Ajuste de range para var_smoothing
    },
    {  # Parâmetros para DecisionTreeClassifier
        'classifier': [DecisionTreeClassifier(random_state=12)],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__splitter': ['best', 'random'],
        'classifier__max_depth': [None, 2, 4, 6, 8, 10, 12],
        'classifier__min_samples_split': [2, 5, 10, None],
        'classifier__min_samples_leaf': [1, 2, 5, 10, None],
        'classifier__max_features': [None, 'sqrt', 'log2'],
    }
]

### Definindo melhores parâmetros com GridSearchCV

In [ ]:
# Variáveis para armazenar os melhores resultados para cada classificador
best_results = {
    'KNeighborsClassifier': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None},
    'GaussianNB': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None},
    'DecisionTreeClassifier': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None}
}

# Dividir o conjunto de dados
X_train, X_test, y_train, y_test = train_test_split(df_atributo, df_resposta, test_size=0.25, random_state=42)

# Iterar sobre cada classificador e grid de hiperparâmetros
for grid in param_grid:

    # Ajustar o GridSearchCV
    grid_search = GridSearchCV(pipeline, grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Prever no conjunto de teste
    y_pred = grid_search.predict(X_test)

    # Calcular a acurácia
    score = accuracy_score(y_test, y_pred)

    # Identificar o classificador utilizado
    classifier_name = grid_search.best_estimator_.named_steps['classifier'].__class__.__name__

    # Verificar se esta iteração tem o melhor resultado para este classificador
    if score > best_results[classifier_name]['best_score']:
        best_results[classifier_name]['best_score'] = score
        best_results[classifier_name]['best_test_size'] = 0.25
        best_results[classifier_name]['best_parameters'] = grid_search.best_params_


### Vendo melhores parâmetros

In [ ]:
# Exibir os melhores resultados por modelo
for classifier, result in best_results.items():
    print(f"{classifier}: Acurácia = {result['best_score']}, \nParâmetros = {result['best_parameters']}\n")

## Balanceando classes com SMOTE

In [ ]:
# Aplicando o smote
smote = SMOTE(random_state=42, k_neighbors=1)
df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)

#Descobindo o tamanho da base
print(f'Tamanho da base com SMOTE: {len(df_atributo)}')

## Separando dados entre treino e teste

In [ ]:
# Divida os dados em conjuntos de treino e teste
df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(df_atributo, df_resposta, test_size=0.25, random_state=42)

# Converta para arrays
atributos_treino = df_atributos_treino.values
resposta_treino = df_resposta_treino.values
atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste.values

## Modelos

### DecisionTreeClassifier

#### Gini

##### Testando com valores de teste

In [ ]:
classificador_tree = tree.DecisionTreeClassifier(
        criterion='gini', 
        min_samples_leaf= 1,
        min_samples_split= 2,
        splitter= 'best'
    )

classificador_tree.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de teste
previsoes = classificador_tree.predict(atributos_teste)

# Avalie o desempenho do modelo
acuracia = accuracy_score(resposta_teste, previsoes)
print(f'Acurácia no conjunto de teste: {acuracia:.4f}')

report_classificacao = classification_report(resposta_teste, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_teste, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)

plt.figure(figsize=(30,7))
tree.plot_tree(classificador_tree, feature_names=df_atributo.columns, class_names=classificador_tree.classes_, filled=True)
plt.show()

##### Testando com valores de treino

In [ ]:
classificador_tree = tree.DecisionTreeClassifier(
        criterion='gini', 
        min_samples_leaf= 1,
        min_samples_split= 2,
        splitter= 'best'
    )

classificador_tree.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de treino
previsoes = classificador_tree.predict(atributos_treino)

# Avalie o desempenho do modelo
acuracia = accuracy_score(resposta_treino, previsoes)
print(f'Acurácia no conjunto de treino: {acuracia:.4f}')

report_classificacao = classification_report(resposta_treino, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_treino, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)

plt.figure(figsize=(30,7))
tree.plot_tree(classificador_tree, feature_names=df_atributo.columns, class_names=classificador_tree.classes_, filled=True)
plt.show()

### KNeighborsClassifier

##### Testando com valores de teste

In [ ]:
classificador_knn = KNeighborsClassifier(n_neighbors= 1)

classificador_knn.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de teste
previsoes = classificador_knn.predict(atributos_teste)

# Avaliação do modelo
taxa_de_acerto = accuracy_score(resposta_teste, previsoes)
print("Taxa de acerto:", round(taxa_de_acerto * 100, 2))

report_classificacao = classification_report(resposta_teste, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_teste, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)

##### Testando com valores de treino

In [ ]:
classificador_knn = KNeighborsClassifier(n_neighbors= 1)

classificador_knn.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de treino
previsoes = classificador_knn.predict(atributos_treino)

# Avaliação do modelo
taxa_de_acerto = accuracy_score(resposta_treino, previsoes)
print("Taxa de acerto:", round(taxa_de_acerto * 100, 2))

report_classificacao = classification_report(resposta_treino, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_treino, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)

### GaussianNB

##### Testando com valores de teste

In [ ]:

classificador_bayes = GaussianNB(var_smoothing= 1.0)

classificador_bayes.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de teste
previsoes = classificador_bayes.predict(atributos_teste)

# Avaliação do modelo
taxa_de_acerto = accuracy_score(resposta_teste, previsoes)
print("Taxa de acerto:", round(taxa_de_acerto * 100, 2))

report_classificacao = classification_report(resposta_teste, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_teste, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)

##### Testando com valores de treino

In [ ]:

classificador_bayes = GaussianNB(var_smoothing= 1.0)

classificador_bayes.fit(atributos_treino, resposta_treino)

# Faça previsões no conjunto de treino
previsoes = classificador_bayes.predict(atributos_treino)

# Avaliação do modelo
taxa_de_acerto = accuracy_score(resposta_treino, previsoes)
print("Taxa de acerto:", round(taxa_de_acerto * 100, 2))

report_classificacao = classification_report(resposta_treino, previsoes)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_treino, previsoes)
print("Matriz de Confusão:")
print(matriz_de_confusao)